In [67]:
import numpy as np
from sklearn.model_selection import train_test_split
import json
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification


In [68]:
# Load data from JSON file
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    sentences = [item['request'] for item in data]  # Extracting sentences from the 'request' key
    labels = [np.argmax([item[key] for key in ['is_order', 'is_refund', 'is_status', 'is_date_change', 'is_dest_change', 'is_weather', 'is_allowed']]) for item in data]
    return sentences, np.array(labels)  # Return sentences as a list of strings





In [69]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")


In [70]:
# Load data
sentences, labels = load_data("data/data.json")


In [71]:
# Tokenize and encode data
max_length = 64  # Adjust according to your data and BERT model's maximum input length
encoded_data = tokenizer(sentences, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

# Convert BatchEncoding to dictionary
encoded_data = {key: encoded_data[key].numpy() for key in encoded_data}

In [72]:
# Convert labels to numpy array
# labels = np.array(labels)
# print(labels)
# Convert one-hot encoded labels to integer indices


In [73]:
#split the data into training and testing
train_inputs, test_inputs, train_labels, test_labels = train_test_split(encoded_data['input_ids'], labels, test_size=0.2, random_state=42)


In [74]:
# Load pre-trained BERT model
num_classes = 7 
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_classes)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [75]:
# Compile the model with sparse categorical cross-entropy loss
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [76]:
# Train the model
batch_size = 16
epochs = 10 # Adjust according to your training needs


history = model.fit(
    train_inputs,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1
)


Epoch 1/10
6/6 [==============================] - 150s 18s/step - loss: 1.9712 - accuracy: 0.1222 - val_loss: 1.9791 - val_accuracy: 0.2000
Epoch 2/10
6/6 [==============================] - 48s 8s/step - loss: 1.9447 - accuracy: 0.1333 - val_loss: 1.9790 - val_accuracy: 0.2000
Epoch 3/10
6/6 [==============================] - 31s 5s/step - loss: 1.9374 - accuracy: 0.1444 - val_loss: 1.9739 - val_accuracy: 0.3000
Epoch 4/10
6/6 [==============================] - 29s 5s/step - loss: 1.8725 - accuracy: 0.2000 - val_loss: 1.8426 - val_accuracy: 0.3000
Epoch 5/10
6/6 [==============================] - 27s 4s/step - loss: 1.6852 - accuracy: 0.4000 - val_loss: 1.5349 - val_accuracy: 0.4000
Epoch 6/10
6/6 [==============================] - 26s 4s/step - loss: 1.4800 - accuracy: 0.6000 - val_loss: 1.2118 - val_accuracy: 0.6000
Epoch 7/10
6/6 [==============================] - 27s 4s/step - loss: 1.1023 - accuracy: 0.6778 - val_loss: 0.8806 - val_accuracy: 0.8000
Epoch 8/10
6/6 [================

In [77]:
#test model accuracy
test_loss, test_accuracy = model.evaluate(test_inputs, test_labels)
print(f"Test accuracy: {test_accuracy}")

1/1 [==============================] - 2s 2s/step - loss: 0.8365 - accuracy: 0.7600
Test accuracy: 0.7599999904632568


In [78]:
# Define directory to save the model
output_dir = "./saved_model"

# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model saved successfully to:", output_dir)



Model saved successfully to: ./saved_model
